### Install Mediapipe

In [1]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled prot

### Helper Func

In [1]:
def calculate_angle(a, b, c):
    """
    세 개의 랜드마크(a, b, c)를 받아 b 지점의 각도와 신뢰도(가시성 곱)를 반환합니다.
    """
    reliability = a[3] * b[3] * c[3]

    if a[3] < 0.5 or b[3] < 0.5 or c[3] < 0.5:
        return None, reliability

    a_xyz = a[:3]
    b_xyz = b[:3]
    c_xyz = c[:3]

    try:
        ba = a_xyz - b_xyz
        bc = c_xyz - b_xyz

        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
        angle = np.arccos(cosine_angle)

        return np.degrees(angle), reliability
    except (ZeroDivisionError, RuntimeWarning):
        return None, reliability

def calculate_plane_normal(a, b, c):
    """
    세 개의 랜드마크(a, b, c)로 정의된 평면의 법선 벡터(Normal Vector)를 반환합니다.
    반환값: [x, y, z] (정규화된 벡터), reliability
    순서는 a -> b -> c 순으로 오른손 법칙을 따릅니다.
    """
    # 신뢰도 계산
    reliability = a[3] * b[3] * c[3]

    # 가시성이 낮은 랜드마크가 포함되면 계산 불가
    if a[3] < 0.5 or b[3] < 0.5 or c[3] < 0.5:
        return None, reliability

    a_xyz = a[:3]
    b_xyz = b[:3]
    c_xyz = c[:3]

    try:
        # 두 개의 벡터 생성 (a -> b, a -> c)
        v1 = b_xyz - a_xyz
        v2 = c_xyz - a_xyz

        # 외적(Cross Product)을 통해 법선 벡터 계산
        normal = np.cross(v1, v2)

        # 벡터 정규화 (크기를 1로 만듦)
        norm_length = np.linalg.norm(normal)
        if norm_length == 0:
            return None, reliability

        normalized_normal = normal / norm_length

        return normalized_normal, reliability

    except (RuntimeWarning, Exception):
        return None, reliability


## Feature Engineering

In [17]:
import numpy as np

# 헬퍼 함수들은 외부에 정의되어 있다고 가정 (calculate_angle, calculate_plane_normal)
# 만약 없다면 맨 처음 드린 코드의 헬퍼 함수를 그대로 쓰시면 됩니다.

def feature_engineering(landmarks):
    """
    [Hybrid Feature Engineering]
    1. Raw 좌표: 어깨 중심 + 어깨 너비 정규화 (Scale Invariant)
    2. 기하학적 특징: 각도(0~1 정규화) + 법선 벡터
    3. 신뢰도 정보 포함

    모든 피처의 스케일을 대략 -1~1 또는 0~1로 맞춰서 결합합니다.
    """
    if landmarks is None: return None

    # ==========================================
    # 1. Raw Coordinates Processing (어깨 중심 & 크기 보정)
    # ==========================================
    lm_copy = landmarks.copy()

    # 인덱스
    L_SHOULDER, R_SHOULDER = 11, 12

    # 기준점(Origin): 어깨 중점
    left_sh = lm_copy[L_SHOULDER, :3]
    right_sh = lm_copy[R_SHOULDER, :3]
    center_point = (left_sh + right_sh) / 2.0

    # 크기 기준(Scale): 어깨 너비
    shoulder_width = np.linalg.norm(left_sh - right_sh)
    scale_factor = shoulder_width if shoulder_width > 1e-6 else 1.0

    # (A) 위치 이동 및 스케일링 -> 결과범위: 대략 -1.0 ~ 1.0
    lm_copy[:, :3] -= center_point
    lm_copy[:, :3] /= scale_factor

    # 1차원으로 펴기 (Visibility 포함 132차원)
    # [Tip] 만약 좌표의 visibility를 굳이 또 넣을 필요 없다면 [:, :3]만 써도 됩니다.
    # 여기서는 정보 손실 방지를 위해 다 넣겠습니다.
    raw_features = lm_copy.flatten()

    # ==========================================
    # 2. Geometric Features (각도 & 법선)
    # ==========================================
    # 인덱스 정의
    NOSE = 0; L_EYE, R_EYE = 2, 5; L_EAR, R_EAR = 7, 8
    L_ELBOW, R_ELBOW = 13, 14; L_WRIST, R_WRIST = 15, 16
    L_HIP, R_HIP = 23, 24

    angle_feats = []
    normal_feats = []
    reliability_feats = []

    # (B) 각도 계산
    angles_to_calc = [
        (landmarks[L_SHOULDER], landmarks[L_ELBOW], landmarks[L_WRIST]),
        (landmarks[R_SHOULDER], landmarks[R_ELBOW], landmarks[R_WRIST]),
        (landmarks[L_ELBOW], landmarks[L_SHOULDER], landmarks[L_HIP]),
        (landmarks[R_ELBOW], landmarks[R_SHOULDER], landmarks[R_HIP]),
        (landmarks[L_EAR], landmarks[L_SHOULDER], landmarks[L_HIP]), # 거북목 확인용
        (landmarks[R_EAR], landmarks[R_SHOULDER], landmarks[R_HIP])
    ]

    for p1, p2, p3 in angles_to_calc:
        ang, rel = calculate_angle(p1, p2, p3)
        # [중요] 각도 정규화: 0~180 -> 0.0~1.0
        if ang is not None:
            angle_feats.append(ang / 180.0)
        else:
            angle_feats.append(0.0)
        reliability_feats.append(rel)

    # (C) 법선 벡터 계산 (이미 -1~1 범위)
    # Face Normal
    face_norm, face_rel = calculate_plane_normal(landmarks[NOSE], landmarks[L_EYE], landmarks[R_EYE])
    if face_norm is not None: normal_feats.extend(face_norm)
    else: normal_feats.extend([0.0, 0.0, 0.0])
    reliability_feats.append(face_rel)

    # Chest/Shoulder Normal
    chest_norm, chest_rel = calculate_plane_normal(landmarks[L_SHOULDER], landmarks[R_SHOULDER], landmarks[NOSE])
    if chest_norm is not None: normal_feats.extend(chest_norm)
    else: normal_feats.extend([0.0, 0.0, 0.0])
    reliability_feats.append(chest_rel)

    # ==========================================
    # 3. Feature Concatenation (결합)
    # ==========================================
    # 리스트들을 numpy array로 변환
    geo_vector = np.array(angle_feats + normal_feats + reliability_feats)

    # [최종 결합] Raw 좌표 벡터 + 기하학적 특징 벡터
    # 차원 수: 132 (Raw) + 6 (Angles) + 6 (Normals) + 8 (Reliability) = 152차원
    final_features = np.concatenate([raw_features, geo_vector])

    return final_features

## MLP(val_accuracy = 0.9613)

In [18]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

def build_model(input_shape):
    """
    특징 벡터를 입력받는 Keras 분류 모델을 구성합니다.
    """
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        # 이진 분류를 위한 Sigmoid 활성화 함수
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')])
    return model

# --- 메인 실행 로직 ---
if __name__ == "__main__":

    # --- 1. 데이터 로드 ---
    # .npy 파일 경로
    X_DATA_PATH = 'X_train.npy'
    Y_DATA_PATH = 'Y_train.npy'
    X_VAL_PATH = 'X_val.npy'
    Y_VAL_PATH = 'Y_val.npy'
    X_TEST_PATH = 'X_test.npy'
    Y_TEST_PATH = 'Y_test.npy'

    print(f"'{X_DATA_PATH}'와 '{Y_DATA_PATH}'에서 데이터셋 로드 중...")

    if not os.path.exists(X_DATA_PATH) or not os.path.exists(Y_DATA_PATH):
        print("="*50)
        print(f"오류: '{X_DATA_PATH}' 또는 '{Y_DATA_PATH}' 파일을 찾을 수 없습니다.")
        print("데이터셋 파일이 스크립트와 동일한 디렉토리에 있는지 확인해주세요.")
        print("="*50)
    else:
        try:
            # .npy 파일에서 각각 랜드마크와 레이블을 불러옵니다.
            raw_landmarks = np.load(X_DATA_PATH)
            raw_val = np.load(X_VAL_PATH)
            raw_test = np.load(X_TEST_PATH)
            labels = np.load(Y_DATA_PATH)
            labels_val = np.load(Y_VAL_PATH)
            labels_test = np.load(Y_TEST_PATH)
            print(f"로드 완료. 랜드마크: {raw_landmarks.shape}, 레이블: {labels.shape}")

            if raw_landmarks.shape[0] == 0:
                print("오류: 로드된 데이터셋이 비어있습니다.")
            else:
                # --- 2. 특징 공학 (Feature Engineering) ---

                print("\n특징 공학(Feature Engineering) 적용 중...")
                features = []
                valid_labels = []

                for i, lm in enumerate(raw_landmarks):
                    feature_vector = feature_engineering(lm)
                    # 특징 추출에 성공한(None이 아닌) 샘플만 사용
                    if feature_vector is not None:
                        features.append(feature_vector)
                        valid_labels.append(labels[i])
                features_val = []
                valid_labels_val = []

                for i, lm in enumerate(raw_val):
                    feature_vector = feature_engineering(lm)
                    # 특징 추출에 성공한(None이 아닌) 샘플만 사용
                    if feature_vector is not None:
                        features_val.append(feature_vector)
                        valid_labels_val.append(labels_val[i])

                features_test = []
                valid_labels_test = []

                for i, lm in enumerate(raw_test):
                    feature_vector = feature_engineering(lm)
                    # 특징 추출에 성공한(None이 아닌) 샘플만 사용
                    if feature_vector is not None:
                        features_test.append(feature_vector)
                        valid_labels_test.append(labels_test[i])

                # X: (n_valid_samples, n_features)
                # y: (n_valid_samples,)
                X_train = np.array(features)
                y_train = np.array(valid_labels)
                X_val = np.array(features_val)
                y_val = np.array(valid_labels_val)
                X_test = np.array(features_test)
                y_test = np.array(valid_labels_test)

                if X_train.shape[0] == 0:
                     print("오류: 모든 샘플에서 특징 공학(각도 계산 등)에 실패했습니다.")
                     print("데이터셋의 가시성(visibility) 값을 확인해주세요.")
                else:

                    print(f"학습 데이터: {X_train.shape[0]}개 / 테스트 데이터: {X_test.shape[0]}개")

                    # --- 4. 모델 학습 ---
                    # 1. 인덱스 배열 생성 (0부터 N-1까지)
                    indices = np.arange(X_train.shape[0])

                    # 2. 인덱스 무작위 섞기
                    np.random.shuffle(indices)

                    # 3. 섞인 인덱스로 X와 y 재배열 (짝 유지됨)
                    X_train = X_train[indices]
                    y_train = y_train[indices]

                    # 입력 차원(input_shape)은 특징 벡터의 크기입니다. (여기서는 8)
                    model = build_model(input_shape=X_train.shape[1])
                    model.summary()

                    print("\n모델 학습 시작...")
                    history = model.fit(
                        X_train,
                        y_train,
                        epochs=500,
                        batch_size=16,
                        validation_data=(X_val, y_val),
                        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)]
                    )

                    print("모델 학습 완료.")

                    # --- 5. 모델 평가 ---
                    loss, accuracy = model.evaluate(X_test, y_test)
                    print(f"\n테스트 데이터 평가 결과:")
                    print(f"Loss (손실): {loss:.4f}")
                    print(f"Accuracy (정확도): {accuracy:.4f}")

                    # --- 6. 모델 저장 ---
                    model.save("All_pose_classifier_keras.h5")
                    print("학습된 모델을 'All_pose_classifier_keras.h5'로 저장했습니다.")

        except Exception as e:
            print(f"오류: 데이터셋 로드 또는 처리 중 예외 발생: {e}")




'X_train.npy'와 'Y_train.npy'에서 데이터셋 로드 중...
로드 완료. 랜드마크: (2139, 33, 4), 레이블: (2139,)

특징 공학(Feature Engineering) 적용 중...
학습 데이터: 2139개 / 테스트 데이터: 250개


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 64)             │         9,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,905 (46.50 KB)

 Trainable params: 11,905 (46.50 KB)

 Non-trainable params: 0 (0.00 B)


모델 학습 시작...
Epoch 1/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.6151 - loss: 0.6671 - val_accuracy: 0.8962 - val_loss: 0.4505
Epoch 2/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6865 - loss: 0.5424 - val_accuracy: 0.9811 - val_loss: 0.2710
Epoch 3/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7369 - loss: 0.4885 - val_accuracy: 0.9670 - val_loss: 0.2965
Epoch 4/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7718 - loss: 0.4348 - val_accuracy: 0.9434 - val_loss: 0.2869
Epoch 5/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7974 - loss: 0.4016 - val_accuracy: 0.9858 - val_loss: 0.1261
Epoch 6/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8316 - loss: 0.3749 - val_accuracy: 0.9953 - val_loss: 0.1184
Epoch 7/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8425 - loss: 0.3363 - val_accuracy: 0.5566 - val_loss: 0.6443
Epoch 8/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8415 - loss: 0


테스트 데이터 평가 결과:
Loss (손실): 0.1452
Accuracy (정확도): 0.9200
학습된 모델을 'All_pose_classifier_keras.h5'로 저장했습니다.


## RandomForest (val_accuracy = 0.8688)

In [19]:
import numpy as np
import os
import joblib  # scikit-learn 모델 저장용
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# --- 메인 실행 로직 ---
if __name__ == "__main__":
    X_DATA_PATH = 'X_train.npy'; Y_DATA_PATH = 'Y_train.npy'
    X_TEST_PATH = 'X_test.npy'; Y_TEST_PATH = 'Y_test.npy'

    if not os.path.exists(X_DATA_PATH):
        print("데이터 파일 없음"); exit()

    try:
        raw_landmarks = np.load(X_DATA_PATH)
        labels = np.load(Y_DATA_PATH)
        raw_test = np.load(X_TEST_PATH)
        labels_test = np.load(Y_TEST_PATH)

        # 데이터 처리 함수
        def process_data(raw_x, raw_y):
            processed_x, processed_y = [], []
            for i, lm in enumerate(raw_x):
                fv = feature_engineering(lm)
                if fv is not None:
                    processed_x.append(fv)
                    processed_y.append(raw_y[i])
            return np.array(processed_x), np.array(processed_y)

        print("데이터 전처리 중...")
        X_train, y_train = process_data(raw_landmarks, labels)
        X_test, y_test = process_data(raw_test, labels_test)

        # --- 셔플링 (Shuffling) ---
        print("데이터 셔플링 중...")
        indices = np.arange(X_train.shape[0])
        np.random.shuffle(indices)
        X_train = X_train[indices]
        y_train = y_train[indices]

        # --- Random Forest 모델 학습 ---
        print(f"\n[Random Forest] 학습 시작 (데이터 개수: {len(X_train)}개)...")
        # n_estimators: 트리의 개수 (보통 100~500)
        rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
        rf_model.fit(X_train, y_train)
        print("학습 완료.")

        # --- 모델 평가 ---
        y_pred = rf_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        print(f"\n테스트 데이터 평가 결과:")
        print(f"Accuracy (정확도): {accuracy:.4f}")
        print("\n분류 보고서:")
        print(classification_report(y_test, y_pred))

        # --- 모델 저장 ---
        joblib.dump(rf_model, "All_pose_classifier_rf.pkl")
        print("모델 저장 완료: 'All_pose_classifier_rf.pkl'")

    except Exception as e:
        print(f"오류 발생: {e}")

데이터 전처리 중...
데이터 셔플링 중...

[Random Forest] 학습 시작 (데이터 개수: 2139개)...
학습 완료.

테스트 데이터 평가 결과:
Accuracy (정확도): 0.9040

분류 보고서:
              precision    recall  f1-score   support

           0       1.00      0.82      0.90       133
           1       0.83      1.00      0.91       117

    accuracy                           0.90       250
   macro avg       0.91      0.91      0.90       250
weighted avg       0.92      0.90      0.90       250

모델 저장 완료: 'All_pose_classifier_rf.pkl'


## Support Vector Machine (val_accuracy = 0.9210)

In [24]:
import numpy as np
import os
import joblib
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report


# --- 메인 실행 로직 ---
if __name__ == "__main__":
    X_DATA_PATH = 'X_train.npy'; Y_DATA_PATH = 'Y_train.npy'
    X_TEST_PATH = 'X_test.npy'; Y_TEST_PATH = 'Y_test.npy'

    if not os.path.exists(X_DATA_PATH):
        print("데이터 파일 없음"); exit()

    try:
        raw_landmarks = np.load(X_DATA_PATH)
        labels = np.load(Y_DATA_PATH)
        raw_test = np.load(X_TEST_PATH)
        labels_test = np.load(Y_TEST_PATH)

        def process_data(raw_x, raw_y):
            processed_x, processed_y = [], []
            for i, lm in enumerate(raw_x):
                fv = feature_engineering(lm)
                if fv is not None:
                    processed_x.append(fv)
                    processed_y.append(raw_y[i])
            return np.array(processed_x), np.array(processed_y)

        print("데이터 전처리 중 (신뢰도 포함)...")
        X_train, y_train = process_data(raw_landmarks, labels)
        X_test, y_test = process_data(raw_test, labels_test)

        print(f"특징 벡터 차원: {X_train.shape[1]}")

        # --- 셔플링 (Shuffling) ---
        print("데이터 셔플링 중...")
        indices = np.arange(X_train.shape[0])
        np.random.shuffle(indices)
        X_train = X_train[indices]
        y_train = y_train[indices]

        # --- SVM 모델 정의 ---
        print(f"\n[SVM] 학습 준비 (데이터 개수: {len(X_train)}개)...")

        # SVM 파이프라인 (스케일링 + SVM)
        svm_model = make_pipeline(
            StandardScaler(),
            SVC(kernel='rbf', C=1.0, probability=True, random_state=42)
        )

        # --- 1. 교차 검증 (Validation) ---
        print("교차 검증(Cross Validation) 진행 중...")
        val_scores = cross_val_score(svm_model, X_train, y_train, cv=5)
        print(f" >> 5-Fold 검증 정확도: {val_scores}")
        print(f" >> 평균 검증 정확도: {np.mean(val_scores):.4f}")

        # --- 2. 전체 데이터로 최종 학습 ---
        print("최종 모델 학습 중...")
        svm_model.fit(X_train, y_train)

        # --- 모델 평가 ---
        y_pred = svm_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        print(f"\n테스트 데이터 평가 결과:")
        print(f"Accuracy (정확도): {accuracy:.4f}")
        print("\n분류 보고서:")
        print(classification_report(y_test, y_pred))

        # --- 모델 저장 ---
        joblib.dump(svm_model, "All_pose_classifier_svm.pkl")
        print("모델 저장 완료: 'All_pose_classifier_svm.pkl'")

    except Exception as e:
        print(f"오류 발생: {e}")

데이터 전처리 중 (신뢰도 포함)...
특징 벡터 차원: 152
데이터 셔플링 중...

[SVM] 학습 준비 (데이터 개수: 2139개)...
교차 검증(Cross Validation) 진행 중...
 >> 5-Fold 검증 정확도: [0.92757009 0.92757009 0.93691589 0.95560748 0.96487119]
 >> 평균 검증 정확도: 0.9425
최종 모델 학습 중...

테스트 데이터 평가 결과:
Accuracy (정확도): 0.6720

분류 보고서:
              precision    recall  f1-score   support

           0       1.00      0.38      0.55       133
           1       0.59      1.00      0.74       117

    accuracy                           0.67       250
   macro avg       0.79      0.69      0.65       250
weighted avg       0.81      0.67      0.64       250

모델 저장 완료: 'All_pose_classifier_svm.pkl'


## Log_Regression(val_accuracy = 0.5708)

In [23]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import mediapipe as mp
import cv2

X_TRAIN_PATH = 'X_train.npy'
Y_TRAIN_PATH = 'Y_train.npy'
X_TEST_PATH = 'X_val.npy'
Y_TEST_PATH = 'Y_val.npy'

raw_landmarks = np.load(X_TRAIN_PATH)
labels = np.load(Y_TRAIN_PATH)
test_landmarks = np.load(X_TEST_PATH)
test_labels = np.load(Y_TEST_PATH)

features = []
valid_labels = []
test_features = []
testlabels = []

for i, lm in enumerate(raw_landmarks):
    feature_vector = feature_engineering(lm)
    # 특징 추출에 성공한(None이 아닌) 샘플만 사용
    if feature_vector is not None:
        features.append(feature_vector)
        valid_labels.append(labels[i])

for i, lm in enumerate(test_landmarks):
    feature_vector = feature_engineering(lm)
    # 특징 추출에 성공한(None이 아닌) 샘플만 사용
    if feature_vector is not None:
        test_features.append(feature_vector)
        testlabels.append(test_labels[i])

X_train = np.array(features)
Y_train = np.array(valid_labels)

X_test = np.array(test_features)
Y_test = np.array(testlabels)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

log_reg_model = LogisticRegression(solver='liblinear', random_state=42)
log_reg_model.fit(X_train_scaled, Y_train)

print("✅ 모델 학습 및 환경 준비 완료")

# --- 2. [추가] 모델 검증 코드 ---
print("\n--- 🧪 모델 검증 결과 (Test Set) ---")
Y_pred = log_reg_model.predict(X_test_scaled)

# 정확도 (Accuracy)
accuracy = accuracy_score(Y_test, Y_pred)
print(f"✅ 정확도: {accuracy * 100:.2f}%")

# 혼동 행렬 (Confusion Matrix)
print("\n[ 혼동 행렬 (Confusion Matrix) ]")
print(confusion_matrix(Y_test, Y_pred))

# 분류 리포트 (Precision, Recall, F1-score)
# target_names는 0번 클래스와 1번 클래스의 이름을 지정합니다.
print("\n[ 분류 리포트 (Classification Report) ]")
print(classification_report(Y_test, Y_pred, target_names=['Bad Posture (0)', 'Good Posture (1)']))
print("-----------------------------------")

✅ 모델 학습 및 환경 준비 완료

--- 🧪 모델 검증 결과 (Test Set) ---
✅ 정확도: 94.81%

[ 혼동 행렬 (Confusion Matrix) ]
[[110   0]
 [ 11  91]]

[ 분류 리포트 (Classification Report) ]
                  precision    recall  f1-score   support

 Bad Posture (0)       0.91      1.00      0.95       110
Good Posture (1)       1.00      0.89      0.94       102

        accuracy                           0.95       212
       macro avg       0.95      0.95      0.95       212
    weighted avg       0.95      0.95      0.95       212

-----------------------------------
